In [1]:
import pandas as pd

# Load the dataset
file_path = 'SampleData.csv'
df = pd.read_csv(file_path)


In [ ]:
df['LongDescription','Title','Tagline','AdditionalDetails']

KeyError: ('LongDescription', 'Title', 'Tagline', 'AdditionalDetails')

,UID,StatusFlag,ViewState,CreateTime,SourceSite,ScrapeTime,ListingNumber,Title_URL,Title,Tagline,...,Attachment_2,Attachment_3,Attachment_4,Attachment_5,BrokerName,BrokerEmail,BrokerNumber,FirmName,BrokerHrefLink,Original_URL
0,527283,Data Received,Hidden,28:59.9,BusinessBroker.Net,43:13.1,527283,https://www.businessbroker.net/business-for-sa...,Award Winning Robust High End Benefits Software,NaN,...,NaN,NaN,NaN,NaN,Rick Carlson,NaN,NaN,NaN,NaN,NaN
1,529448,Data Received,Hidden,28:45.1,BusinessBroker.Net,39:28.3,529448,https://www.businessbroker.net/business-for-sa...,SaaS Provider/Recurring Revenue/New Media,NaN,...,NaN,NaN,NaN,NaN,Susan Rosner,NaN,NaN,NaN,NaN,NaN
2,594439,Data Received,Hidden,21:45.3,BusinessBroker.Net,26:57.9,594439,https://www.businessbroker.net/business-for-sa...,Commission Processing Platform,NaN,...,NaN,NaN,NaN,NaN,John Geiwitz,NaN,NaN,NaN,NaN,NaN
3,598254,Data Received,Hidden,20:48.0,BusinessBroker.Net,45:01.7,598254,https://www.businessbroker.net/business-for-sa...,Unrivaled Software Tech For VAR Hardware Resel...,NaN,...,NaN,NaN,NaN,NaN,Rick Carlson,NaN,NaN,NaN,NaN,NaN
4,614266,Data Received,Hidden,05:20.0,BusinessBroker.Net,29:52.6,614266,https://www.businessbroker.net/business-for-sa...,Cloud Automation SAAS Company 115% Net Revenue,NaN,...,NaN,NaN,NaN,NaN,Ashish Bhatt,NaN,NaN,NaN,NaN,NaN
